In [38]:
import torchfile
import numpy as np
import time, sys
import os

from keras.regularizers import l2
from keras.models import Sequential
from keras.layers import Convolution2D, Dense, Flatten, Activation, MaxPooling2D, Permute, AveragePooling2D
from keras.constraints import nonneg
from keras.utils.np_utils import to_categorical
from keras import optimizers
from keras.layers.advanced_activations import LeakyReLU

# Load Data

In [2]:
train_data = torchfile.load('/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/trainobjcat.t7')
test_data = torchfile.load('/home/jaichitra.balakrishnan/dl_project/DSCI6005-FinalProject/data/valobjcat.t7')

# Test and Train Data

In [66]:
X_train = train_data[b'x'][0]
y_train = train_data[b'y']
X_test = test_data[b'x'][0]
y_test = test_data[b'y']

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((6415, 3, 128, 128), (6415,), (1604, 3, 128, 128), (1604,))

In [67]:
X_train=np.swapaxes(np.swapaxes(X_train, 1, 2), 2, 3)
X_test=np.swapaxes(np.swapaxes(X_test, 1, 2), 2, 3)

In [68]:
X_train.shape, X_test.shape

((6415, 128, 128, 3), (1604, 128, 128, 3))

# Model Architecture

In [39]:
nb_filter = 500
model = Sequential()
model.add(Convolution2D(filters=nb_filter, data_format = 'channels_first', padding='same',kernel_size=(7,7), kernel_constraint=nonneg(), kernel_regularizer=l2(1e-3), name='conv', input_shape=X_train[0].shape))
#model.add(Activation('LeakyReLU'))
model.add(LeakyReLU(alpha=.001)) 
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Permute([3, 1, 2]))

model.add(Convolution2D(filters=nb_filter+200, kernel_size=(5,5), kernel_constraint=nonneg(), padding='same', kernel_regularizer=l2(1e-3), name='conv1'))
model.add(Activation('tanh'))
#model.add(LeakyReLU(alpha=.001)) 
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Permute([3, 1, 2]))

model.add(Convolution2D(filters=nb_filter+400, kernel_size=(3,3), padding='same', kernel_constraint=nonneg(), kernel_regularizer=l2(1e-3), name='conv2'))
model.add(Activation('relu'))
model.add(AveragePooling2D(pool_size=(2, 2)))
#model.add(Permute([3, 1, 2]))

model.add(Flatten(name='flatten'))
model.add(Dense(units=282, kernel_regularizer=l2(1e-3), name='softmax_weights'))
model.add(Activation('softmax', name='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv (Conv2D)                (None, 500, 128, 128)     74000     
_________________________________________________________________
leaky_re_lu_26 (LeakyReLU)   (None, 500, 128, 128)     0         
_________________________________________________________________
max_pooling2d_64 (MaxPooling (None, 250, 64, 128)      0         
_________________________________________________________________
conv1 (Conv2D)               (None, 250, 64, 700)      2240700   
_________________________________________________________________
activation_42 (Activation)   (None, 250, 64, 700)      0         
_________________________________________________________________
max_pooling2d_65 (MaxPooling (None, 125, 32, 700)      0         
_________________________________________________________________
conv2 (Conv2D)               (None, 125, 32, 900)      5670900   
__________

In [3]:
DATA_DIR = os.getcwd().rsplit('/', 1)[0] + '/data'
checkpoint_dir = os.getcwd().rsplit('/', 1)[0] + '/checkpoint'